<a href="https://colab.research.google.com/github/kimsojeong1225/DeepLearning/blob/master/Tensorflow_Guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 : 초보자를 위한 빠른 시작

In [0]:
from __future__ import absolute_import,division,print_function,unicode_literals

In [3]:
!pip install -q tensorflow-gpu==2.0.0-rc1
import tensorflow as tf
 

     |████████████████████████████████| 380.5MB 42kB/s 
     |████████████████████████████████| 501kB 50.0MB/s 
     |████████████████████████████████| 4.3MB 54.4MB/s 


In [4]:
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train,x_test=x_train/255.0,x_test/255.0

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
model=tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28,28)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10,activation='softmax')
])

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
model.fit(x_train,y_train,epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 6s 96us/sample - loss: 0.2988 - accuracy: 0.9129
Epoch 2/5
60000/60000 [==============================] - 5s 86us/sample - loss: 0.1440 - accuracy: 0.9578
Epoch 3/5
60000/60000 [==============================] - 5s 84us/sample - loss: 0.1090 - accuracy: 0.9676
Epoch 4/5
60000/60000 [==============================] - 5s 85us/sample - loss: 0.0886 - accuracy: 0.9729
Epoch 5/5
60000/60000 [==============================] - 5s 86us/sample - loss: 0.0763 - accuracy: 0.9759


In [10]:
model.evaluate(x_test,y_test,verbose=2)

10000/1 - 1s - loss: 0.0373 - accuracy: 0.9767


[0.0728471383321099, 0.9767]

# 2 : 전문가를 위한 빠른 시작

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow-gpu==2.0.0-rc1
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [0]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [0]:
x_train=x_train[...,tf.newaxis]
x_test=x_test[...,tf.newaxis]

In [0]:
train_ds=tf.data.Dataset.from_tensor_slices(
    (x_train,y_train)).shuffle(10000).batch(32)
test_ds=tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel,self).__init__()
    self.conv1=Conv2D(32,3,activation='relu')
    self.flatten=Flatten()
    self.d1=Dense(128,activation='relu')
    self.d2=Dense(10,activation='softmax')
    
  def call(self,x):
    x=self.conv1(x)
    x=self.flatten(x)
    x=self.d1(x)
    return self.d2(x)
model=MyModel()

In [0]:
loss_object=tf.keras.losses.SparseCategoricalCrossentropy()
optimizer=tf.keras.optimizers.Adam()

In [0]:
train_loss=tf.keras.metrics.Mean(name='train_loss')
train_accuracy=tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss=tf.keras.metrics.Mean(name='test_loss')
test_accuracy=tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [0]:
@tf.function
def train_step(images,labels):
  with tf.GradientTape() as tape:
    predictions=model(images)
    loss=loss_object(labels,predictions)
  gradients=tape.gradient(loss,model.trainable_variables)
  optimizer.apply_gradients(zip(gradients,model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels,predictions)


In [0]:
@tf.function
def test_step(images,labels):
  predictions=model(images)
  t_loss=loss_object(labels,predictions)

  test_loss(t_loss)
  test_accuracy(labels,predictions)

In [37]:
EPOCHS=5

for epoch in range(EPOCHS):
  for images,labels in train_ds:
    train_step(images,labels)

  for test_images,test_labels in test_ds:
    test_step(test_images,test_labels)
  
  template='에포크:{}, 손실 :{}, 정확도 :{}, 테스트 손실 :{}, 테스트 정확도 :{}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

에포크:1, 손실 :0.13337527215480804, 정확도 :95.99166870117188, 테스트 손실 :0.06438282132148743, 테스트 정확도 :97.91999816894531
에포크:2, 손실 :0.08810966461896896, 정확도 :97.3125, 테스트 손실 :0.0565367266535759, 테스트 정확도 :98.13999938964844
에포크:3, 손실 :0.06600899249315262, 정확도 :97.97721862792969, 테스트 손실 :0.054850902408361435, 테스트 정확도 :98.22333526611328
에포크:4, 손실 :0.05276569724082947, 정확도 :98.37667083740234, 테스트 손실 :0.05330182611942291, 테스트 정확도 :98.30750274658203
에포크:5, 손실 :0.04394517093896866, 정확도 :98.64299774169922, 테스트 손실 :0.05668920278549194, 테스트 정확도 :98.26399993896484
